# กลุ่ม seek101
# เมธาวัฒน์ คูศิริวานิชกร 660710735
# ธนยศ รัตนสุรีย์โรจน์ 660710710
# ธนัฐพิพัฒน์ เขียววิจิตร 660710713

In [1]:
# ติดตั้งผ่าน pip
!pip install duckdb

In [74]:
import duckdb

In [72]:
con = duckdb.connect(database =':memory:', read_only=False)

In [75]:
query = "select * from 'spotify_dataset.csv'"

In [76]:
result = con.execute(query).fetchdf()

In [77]:
result.head()

,Index,Highest Charting Position,Number of Times Charted,Week of Highest Charting,Song Name,Streams,Artist,Artist Followers,Song ID,Genre,...,Danceability,Energy,Loudness,Speechiness,Acousticness,Liveness,Tempo,Duration (ms),Valence,Chord
0,1,1,8,2021-07-23--2021-07-30,Beggin',"48,633,449",Måneskin,3377762,3Wrjm47oTz2sjIgck11l5e,"['indie rock italiano', 'italian pop']",...,0.714,0.8,-4.808,0.0504,0.127,0.359,134.002,211560,0.589,B
1,2,2,3,2021-07-23--2021-07-30,STAY (with Justin Bieber),"47,248,719",The Kid LAROI,2230022,5HCyWlXZPP0y6Gqq8TgA20,['australian hip hop'],...,0.591,0.764,-5.484,0.0483,0.0383,0.103,169.928,141806,0.478,C#/Db
2,3,1,11,2021-06-25--2021-07-02,good 4 u,"40,162,559",Olivia Rodrigo,6266514,4ZtFanR9U6ndgddUvNcjcG,['pop'],...,0.563,0.664,-5.044,0.154,0.335,0.0849,166.928,178147,0.688,A
3,4,3,5,2021-07-02--2021-07-09,Bad Habits,"37,799,456",Ed Sheeran,83293380,6PQ88X9TkUIAUIZJHW2upE,"['pop', 'uk pop']",...,0.808,0.897,-3.712,0.0348,0.0469,0.364,126.026,231041,0.591,B
4,5,5,1,2021-07-23--2021-07-30,INDUSTRY BABY (feat. Jack Harlow),"33,948,454",Lil Nas X,5473565,27NovPIUIRrOZoCHxABJwK,"['lgbtq+ hip hop', 'pop rap']",...,0.736,0.704,-7.409,0.0615,0.0203,0.0501,149.995,212000,0.894,D#/Eb


In [78]:
result.shape

(1556, 23)

In [79]:
result.columns

Index(['Index', 'Highest Charting Position', 'Number of Times Charted',
       'Week of Highest Charting', 'Song Name', 'Streams', 'Artist',
       'Artist Followers', 'Song ID', 'Genre', 'Release Date', 'Weeks Charted',
       'Popularity', 'Danceability', 'Energy', 'Loudness', 'Speechiness',
       'Acousticness', 'Liveness', 'Tempo', 'Duration (ms)', 'Valence',
       'Chord'],
      dtype='object')

#หาศิลปินที่มีผู้ติดตามมากที่สุดลงมา

In [179]:
query = """
SELECT
      "Artist",
    MAX(TRY_CAST("Artist Followers" AS BIGINT)) AS max_followers
FROM read_csv_auto('spotify_dataset.csv')
GROUP BY "Artist"
ORDER BY max_followers DESC
"""

In [180]:
result = con.execute(query).fetchdf()

In [181]:
result.head(10)

,Artist,max_followers
0,Ed Sheeran,83337783
1,Ariana Grande,67208170
2,Drake,56327031
3,"Future, Drake, Young Thug",56327031
4,"Drake, WizKid, Kyla",56308172
5,Justin Bieber,48544923
6,"Justin Bieber, Quavo",48544923
7,"Justin Bieber, Kehlani",48544923
8,Billie Eilish,47061119
9,Eminem,46814751


#หาชื่อศิลปินที่ปล่อยจำนวนเพลงออกมามากที่สุดลงมา

In [182]:
query = """
SELECT
      Artist,
      COUNT("Song Name") as total_song
      FROM read_csv_auto('spotify_dataset.csv')
      GROUP BY "Artist"
      ORDER BY total_song DESC
"""

In [183]:
result = con.execute(query).fetchdf()

In [184]:
result.head(10)

,Artist,total_song
0,Taylor Swift,52
1,Justin Bieber,32
2,Lil Uzi Vert,32
3,Juice WRLD,30
4,BTS,29
5,Pop Smoke,29
6,Bad Bunny,28
7,Eminem,22
8,The Weeknd,21
9,Ariana Grande,20


#หาชื่อศิลปินและชื่อเพลงที่ปล่อยมาปี 2019

In [187]:
query = """
SELECT
    "Artist",
    "Song Name",
    SUBSTR("Release Date", 1, 4) AS year
      FROM read_csv_auto('spotify_dataset.csv')
      WHERE SUBSTR("Release Date", 1, 4) = '2019'
"""

In [188]:
result = con.execute(query).fetchdf()

In [189]:
result.head(10)

,Artist,Song Name,year
0,Harry Styles,Watermelon Sugar,2019
1,Lewis Capaldi,Someone You Loved,2019
2,Tones And I,Dance Monkey,2019
3,"Justin Wellington, Small Jam",Iko Iko (My Bestie),2019
4,Duncan Laurence,Arcade,2019
5,Post Malone,Circles,2019
6,Lewis Capaldi,Before You Go,2019
7,dhruv,double take,2019
8,Doja Cat,Streets,2019
9,"Shawn Mendes, Camila Cabello",Señorita,2019


#หาชื่อศิลปินที่ปล่อยเพลง pop มามากที่สุดลงมา

In [201]:
query = """
SELECT
    "Artist",
    count("Song Name") AS "POP_SONG"
      FROM read_csv_auto('spotify_dataset.csv')
      WHERE LOWER("Genre") LIKE '%pop%'
      GROUP BY "Artist"
      ORDER BY "POP_SONG" DESC
"""

In [202]:
result = con.execute(query).fetchdf()

In [203]:
result.head(10)

,Artist,POP_SONG
0,Taylor Swift,52
1,Justin Bieber,30
2,BTS,29
3,The Weeknd,21
4,Drake,19
5,Ariana Grande,18
6,Doja Cat,16
7,Selena Gomez,16
8,Dua Lipa,13
9,Lady Gaga,13


#หาจำนวนเพลงที่ปล่อยมาในปีต่างๆและชื่อศิลปิน

In [230]:
query = """
SELECT
   Artist,
   SUBSTR("Release Date",1,4) AS year,
   COUNT("Song Name") as total_song_year
FROM read_csv_auto('spotify_dataset.csv')
GROUP BY Artist, SUBSTR("Release Date",1,4)
ORDER BY total_song_year DESC
"""

In [231]:
result = con.execute(query).fetchdf()

In [232]:
result.head(10)

,Artist,year,total_song_year
0,Taylor Swift,2020,32
1,Lil Uzi Vert,2020,30
2,Bad Bunny,2020,25
3,BTS,2020,24
4,Juice WRLD,2020,21
5,Eminem,2020,19
6,Pop Smoke,2020,18
7,Justin Bieber,2020,18
8,Taylor Swift,2021,16
9,The Weeknd,2020,15


#บอกชื่อเพลงกับชื่อศิลปินที่มียอด Streams มากกว่า 40,000,000

In [233]:
query_stream = """
select
  "Song Name",
  "Artist",
  "Streams",
from read_csv_auto('spotify_dataset.csv')
where cast(replace("Streams", ',', '') as bigint) > 40000000
"""
re_stream = con.execute(query_stream).fetchdf()
re_stream

,Song Name,Artist,Streams
0,Beggin',Måneskin,"48,633,449"
1,STAY (with Justin Bieber),The Kid LAROI,"47,248,719"
2,good 4 u,Olivia Rodrigo,"40,162,559"


#บอกชื่อศิลปิน กับวันที่ปล่อย ของเพลงแนว pop

In [234]:
query_pop = """
select
  "Artist",
  "Genre",
  "Release Date",
from read_csv_auto('spotify_dataset.csv')
where Genre like '%pop%'
"""
re_pop = con.execute(query_pop).fetchdf()
re_pop.head(10)

,Artist,Genre,Release Date
0,Måneskin,"['indie rock italiano', 'italian pop']",2017-12-08
1,Olivia Rodrigo,['pop'],2021-05-21
2,Ed Sheeran,"['pop', 'uk pop']",2021-06-25
3,Lil Nas X,"['lgbtq+ hip hop', 'pop rap']",2021-07-23
4,Lil Nas X,"['lgbtq+ hip hop', 'pop rap']",2021-03-31
5,Doja Cat,"['dance pop', 'pop']",2021-04-09
6,Rauw Alejandro,"['puerto rican pop', 'trap latino']",2021-05-20
7,Måneskin,"['indie rock italiano', 'italian pop']",2021-03-19
8,Dua Lipa,"['dance pop', 'pop', 'uk pop']",2020-10-01
9,BTS,"['k-pop', 'k-pop boy group']",2021-07-09


#บอกชื่อศิลปิน ชื่อเพลง และความยาวของเพลง เรียงจากเพลงที่นานที่สุดไปเพลงที่สั้นที่สุด

In [236]:
query_duration = """
select
  "Artist",
  "Song Name",
  "Duration (ms)",
  (try_cast("Duration (ms)" as bigint) / 1000.0) / 60.0 as "duration_minutes"
from read_csv_auto('spotify_dataset.csv')
where
  try_cast("Duration (ms)" as bigint) is not null
order by "duration_minutes" desc
"""
re_duration = con.execute(query_duration).fetchdf()
re_duration.head(10)

,Artist,Song Name,Duration (ms),duration_minutes
0,"Tyler, The Creator",SWEET / I THOUGHT YOU WANTED TO DANCE (feat. B...,588139,9.802317
1,"Tyler, The Creator",WILSHIRE,515865,8.597750
2,Justin Timberlake,Mirrors,484147,8.069117
3,Residente,René,457592,7.626533
4,"J Balvin, KAROL G, Nicky Jam, Crissin, Totoy E...",Poblado - Remix,393280,6.554667
5,"Crissin, Totoy El Frio, Natan & Shander",Poblado,393280,6.554667
6,"Kendrick Lamar, Jay Rock",Money Trees,386907,6.448450
7,Drake,Lemon Pepper Freestyle (feat. Rick Ross),383037,6.383950
8,Tame Impala,Breathe Deeper,372867,6.214450
9,The Weeknd,After Hours,361027,6.017117


#บอกชื่อเพลง และแนวเพลง เรียงตามความดังของเพลง(หน่วยวัด เดซิเบล) จำนวน 5 อันดับ

In [241]:
query_Loud = """
select
  "Song Name",
  "Genre",
  "Loudness"
from read_csv_auto('spotify_dataset.csv')
where
  try_cast("Loudness" as bigint) is not null
order by "Loudness" ASC
limit 5
"""
re_Loud = con.execute(query_Loud).fetchdf()
re_Loud

,Song Name,Genre,Loudness
0,Sal y Perrea,"['latin', 'panamanian pop', 'reggaeton', 'trap...",-0.515
1,Investe Em Mim,"['forro', 'sertanejo', 'sertanejo pop', 'serta...",-1.283
2,eight(Prod.&Feat. SUGA of BTS),['k-pop'],-1.573
3,Cradles,['modern indie pop'],-1.865
4,"Não, Não Vou",[],-1.891


#บอกชื่อศิลปิน ชื่อเพลง แนวเพลง และวันที่ปล่อย เรียงจากเพลงที่มี Chart มากที่สุดไปน้อยที่สุด จำนวน 10 อันดับ

In [242]:
query_chart = """
select
  "Artist"
  "Song Name",
  "Genre",
  "Release Date",
  "Number of Times Charted",
from read_csv_auto('spotify_dataset.csv')
order by "Number of Times Charted" desc
limit 10
"""
re_chart = con.execute(query_chart).fetchdf()
re_chart

,Song Name,Genre,Release Date,Number of Times Charted
0,Harry Styles,"['pop', 'post-teen pop']",2019-12-13,142
1,Post Malone,"['dfw rap', 'melodic rap', 'rap']",2019-09-06,84
2,Lewis Capaldi,"['pop', 'uk pop']",2019-11-22,83
3,Tones And I,"['australian pop', 'pop']",2019-10-17,83
4,Harry Styles,"['pop', 'post-teen pop']",2019-12-13,83
5,Billie Eilish,"['electropop', 'pop']",2018-04-19,83
6,SAINt JHN,"['melodic rap', 'pop rap', 'rap', 'slap house'...",2020-11-20,83
7,Imagine Dragons,"['modern rock', 'pop', 'rock']",2017-06-23,83
8,The Weeknd,"['canadian contemporary r&b', 'canadian pop', ...",2020-03-20,83
9,Lewis Capaldi,"['pop', 'uk pop']",2019-05-17,83
